# IBM Cloud Hyper Protect Secure Build Server and BYOI lab

## This lab is under construction and will probably not run correctly for you as long as this warning message is here!  

This lab will introduce you to two features of the IBM Cloud Hyper Protect Virtual Servers offering:

1. Secure Build Server
2. Bring Your Own Image, or BYOI

The initial introduction of IBM Cloud Hyper Protect Virtual Servers (HPVS) offered an Ubuntu image which you could then tailor to your requirements, but if you wanted to replicate your changes onto other HPVS instances, you would have to repeat the custom configurations on each additional HPVS instance you acquired.  Automation tools like Ansible could help with that process, but wouldn't it be nicer if you could just deploy your custom-built Docker image, that you only had to configure once, onto an HPVS instance?  That's the convenience that the *BYOI* feature offers. It offers the ability to deploy a digitally-signed Docker image into an HPVS instance.

You can deploy Docker images that are digitally-signed by you or some other person or organization that you (hopefully!) trust. In the case of images digitally-signed by you, the _Secure Build Server_ feature lets you build, and digitally sign, a Docker image within an HPVS instance called a _secure build server_ that offers the confidentiality, integrity and auditability protections inherent in IBM Cloud Hyper Protect Virtual Servers.  

In this lab you will perform the main use case targeted by these two features working together:

1. You will deploy your own secure build server in an HPVS instance
2. You will use this secure build server to securely create and digitally sign a Docker image that contains a sample application
3. You will deploy this Docker image that you've built in a second HPVS instance and then run the sample application contained within the image.

In this lab you will be using an open-source application called _Juypter Notebook_ to run the lab. This application allows you to enter commands and see the results of these commands. You are likely reading this within Jupyter Notebook right now.  You will also open a terminal within Jupyter and enter some commands from the terminal. The large majority of the commands will be run from within the Jupyter notebook while a small number of commands will be run from the terminal.

## Open a terminal within Jupyter

You should have two tabs open in your browser- this tab, and the Juypter home page from which your instructors guided you to this tab.

Go back to the Jupyter home page tab, and in the upper right, click the *New* button and then choose *Terminal* to start up a terminal.

From that terminal, enter the command `ibmcloud login --sso` and follow the instructions to log in to your IBM Cloud account.

You will first be given a *Y/n* prompt, reply *Y*.

At that point, the login command tries to open a browser tab, but for our lab, you are running it within a Kubernetes pod that cannot open the browser tab, so it will instruct you to copy the URL into another browser tab- go ahead an do that, and follow any necessary login prompts, and you should eventually be given a page with a login token.  Click on that token to put in your clipboard, and then, paste that into the command prompt.  When successful, your terminal window will look like this, but with your account information of course: 

```

API endpoint:      https://cloud.ibm.com
Region:            us-south
User:              silliman@us.ibm.com
Account:           Barry Silliman's Account (1e963a246cc69a44df65e277e14239d5) <-> 1996902
Resource group:    No resource group targeted, use 'ibmcloud target -g RESOURCE_GROUP'
CF API endpoint:
Org:
Space:
```

When that is complete, continue in this notebook.

You can see from the above output that no "resource group" is targeted when you log in. Run this command in order to target the _default_ resource group. "Resource groups" are logical buckets that you can define in order to categorize your IBM Cloud resources as you wish.  We will simply use the "default" resource group which exists already.

The text you are reading now is in a type of Juypter notebook cell called "markdown".  It is intended for instructions, and includes text and possibly images.

Commands that you will enter (with the exception of the small number of commands, such as the `ibmcloud login --sso` above, that are run in the terminal) will be in a second type of Jupyter notebook cell called "code".  At the top of this page you can see a dropdown box that will show you whether or not you are in a _markdown_ cell or a _code_ cell.

Click on the cell below that contains the command `ibmcloud target -g default`. You should see _code_ in the dropdown box at the top.  Then click the _Run_ button above, and that command will be run, and its output will be shown.

This will target your _default_ resource group.

In [ ]:
! ibmcloud target -g default

## Create an ssh key for the lab

### TODO create instructions for this

## Create an IBM Cloud IAM API Key for this lab

Run the command in the _code_ cell below in order to create an API key for your user ID. This will be used in the lab for you to store your custom-built Docker image into the IBM Cloud Container registry:

*Note:* If you are an experienced IBM Cloud user and already have some API keys, create a new one anyway, for usage just for this lab, and delete it when the lab is over. Do not use it for other purposes. If, by chance, you already have an API key named _myapikey_, choose a different name for this lab.  You can do that by double-clicking on _myapikey_ in the code cell below to highlight it and then change its name before you run the code cell.

In [ ]:
! ibmcloud iam api-key-create myapikey -d "API key for SBS tutorial"

### save the value of the API key

The output of the above command will contain an _API Key_ field.  Copy this and save it someplace safe for the duration of this lab.  (The output should remain within the notebook but save it elsewhere as well in case the output is accidentally cleared. There is no way to retrieve this value later.  The _API Key_ field should be treated as a password, as that is essentially what it is.

## Set your IBM Cloud Container Registry region to us-south

Run the code cell below in order to set your IBM Cloud Container Registry region to us-south

In [ ]:
! ibmcloud cr region-set us-south

## Create a container registry namespace

The code cell below contains a command to add a namespace to the IBM Cloud Container Registry that will be owned by you. You can try to run the command as shown, but it will definitely fail- go ahead and try it, it won't hurt anything.  It will tell you that the _secureimages_ namespace already exists.  Change _secureimages_ to something unique by adding your initials and a number or some other unique string to the namespace.  Keep trying until you get a message that indicates success.

*Note:* If you have used IBM Cloud Container Registry previously, you may already have a namespace. Feel free to either use that or create a new one for this lab.

*Jupyter Notebook tip:* If you click the *Run* button above after you've highlighted the code cell below (or any cell for that matter) it will attempt to run the code and then move to the next cell.  If you need to retry because the name you chose was already taken, just click in the cell again and try again.  Alternatively, you can use the menu and choose *Cell->Run Cell* which will run the command but then the cursor will stay in that same cell. (The *Run* button is the same as if you had chosen *Cell->Run Cells and Select Below* from the menu).

In [ ]:
! ibmcloud cr namespace-add bjs-secureimages

### Download the Secure Build command line interface (CLI):

In order to set up your own Secure Build Server within an HPVS instance, you will use a command line interface (CLI) that is provided by IBM.  This CLI is implemented in Python and is publicly available (including source code) in a GitHub repository, which you will download now into your lab environment when you run the following code cell:

In [ ]:
! git clone git@github.com:ibm-hyper-protect/secure-build-cli.git

*Note:* The code cells of this Jupyter Notebook are all run from the directory in which the notebook resides.  Even if a prior code cell changed the directory (e.g., with the `cd` command) for the rest of that code cell's execution, that directory change doesn't persist into the next code cell.  That is why you will see, for many of the code cells, a `cd secure-build-cli` command at the beginning of the code cell, followed by one or more other commands.  This is the lab author's first foray into Jupyter Notebook, so there may be a way to work around this incovenience that he is currently unaware of.

There are some python packages required by the Secure Build CLI, and running the following code cell will install these packages using *pip3*, the Python 3.x-based package installer:

In [ ]:
! cd secure-build-cli && pip3 install -r requirements.txt

## Create the Secure Build Server registration file

Part of the "BYOI" workflow involves creating a secure registration file, which contains secrets and metadata that can only be decrypted by the running image.  When you build an image later on in the lab, you will download a secure registration file for your image.  

The Secure Build Server itself is deployed securely within an HPVS instance with the help of its own secure registration file.  The code cell below will place this file into your lab environment workspace with the name `secure_build.asc`.

This file is posted publicly on the IBM Cloud documentation website, but the Secure Build CLI repo does not contain it.  That's why you are creating it now.  This file is encrypted, and only the Secure Build Server can decrypt it.  Neither you, nor any IBM Cloud administrator, can access the secure build server (except through the Secure Build CLI), by design, so that the decrypted contents of this file cannot be obtained.

In [ ]:
%%bash
cd secure-build-cli
cat <<EOF > secure_build.asc
 -----BEGIN PGP MESSAGE-----

 hQIMA4AFxsw8YZ3rARAAnR1PB+ij0LNdueQIxtzIVnJ2ogQdh7+akYLC/V85u8Xh
 NKZmxsjwzVsCISZOSwvn/BbylonBPXG+c99gI60ChA+vjpCmXqU7dASWTjUyBfYZ
 piu6ktHRpQrsAPPZl2o+17yjFYnlRpG2FkVTfEMMAdGbC4j+g345RgF3trBgsMdF
 rvaJa2mbsekau9I076hNHKgm02ar3VHET8Y1sMGmij87cgmvKipqoyF/TIfv8C0c
 fK2CX4ty8DRMINPSojGiDTR15JBd+QwdeW9cO1fKEczKMiXiNcDawo8X0r0ICHKs
 +QssrlOwn0XG7QzQhW6DOdAjWmu5QnlCAs5+yJ47vMfLL5g+84S2Bm2Lrn6fbeVc
 cBcBZr79Bbp4uDsF8OMVfrEl2Kv3fQnTbnA3bL7k1W/ABU5ytPYnBZDo/h45xq8z
 Cpw3lVNnMXqN2+i4Dc3dF+qtSMOo3nlmEcMxfhUDooWAyXX4/cysjU9P1TCs4OsA
 inov8GM7V2nYXFTKo9rKfS7UMiqsLAKwpw6YvP2tZUoNk/G8Wyb4cjB052R28MSG
 H8VBbIouKyJhDuSJk4J2pwWCwQYFcE5CTGzIStWZpu5CdYr9TMlVNZJ1jrwp0a+o
 wlFA3WHGYczSWQa0smc8U7L/jcOVAtV0u2Q+5g3AwJUy/ogZ7WfynJ+t7H3a0WjS
 7AGCRvaiPyVroBJuaGOwFHAXE19u5/I4xHgYpDaaQlbcX2m3MVdi8N96UHnTXKvH
 QLPi7IeciT2/56eYULzynb8PuFyQbmS8WSeSJmjUB0wZB12Ys4xUAUObuHwzLgk0
 2zPRJjRM7NrDw824yJGmSDr+I9E30we8TMJX4ZGP2a2q8RrXlxLgUqgMU8QKe+wx
 p96HmaNC3L4Vahhr7/U2OQ638r/ow++xOvbhVn3X8uFmA5I9PW92AdpqtaRuKTtw
 a1+63K32ctcWaEH7DUGfBmWaqpD/QZbE/SFhMT+sHEuLXgOP/dbTdWSGtW4upkGp
 bIwrcU30gZ05J1O8s2kRc+K2m4Ty44AILT0ddhGCGJicJDLAl0JNryFP4Eazs/Q8
 WawQN6sl6Hr7f2QDjJcaxUtZ3uO7/r46AjRap8j+UP3GJEoDf2Ry1/lVO1xGTEWl
 KNo4db5Xkuxq1xNJPCigpv206eH0Fuk8G3S5EK+h7PUmZ1aQgE8GzZJs21FqI+gY
 WGqXqp98rLYFX4IjEm7be8kTWlB0QZCtfA1cqtWNRy0chrak+XYbgPPptai26Eox
 hIAInWEkIzLWC/4t7WIgIOOWwKyMdeNHj0iAlihGToqtmqtcE5pj+CyRqzA7uHUb
 w1H4+q0w92Tcf6d7+xKrZ1OuJZ+5v7j/S1ZhLCMWC0yQzEZCCK4pc8D3I6ZoMcRK
 nlgJXKznjI27A6ZwXi8yBJQu+TvJIwIpP2nTF9LK76bktsQYAAS/zcxSBlCSMkeu
 gqeE82sf32j0JvAdplbHD0uq9fQeJRJQ2nMae7qYwIy6d2XMZMWCKsKxKm9BB8bU
 qpmtGIGjHyTZTCTh/BVsaPL++jCKhLq4W/8J5C/58hhfR4gH/V6sDmmAQb85wotT
 tGnZzVDDqL1zk1Qou7+6tndNdOTWVmydp70xkJE393CngLiuzDCoZuFFC8F3XZXW
 Qx+2+upbmhoqg9QwnMZPjemb1sesMHPUJn+wRlpSbiYiaYA9qxyozuQBcq2C8zP7
 Y8Kpbsajve4dKvXz4p9S1jsRzuE1wQ0/3WjUawYQQiv08v9yurhUB2yWilfv0ypX
 1TrISFaCkHBhDK10m/2VzGDR9lr5hPcHL8KuM3Mi5XqQagvmPrdLC4UR0EP7NveX
 zkg4GkBgrQpV9Sxq9x6zndSymdCpmYVUnBRZtS94qAhfpr3riMkL1EUWvzlVp6um
 MbMBKjWhtBpvLNTeFR2qtE5FjFyeY0k08MT6W5MvtJ4tljOX65kdXDeBa+HZD+GI
 Ae3UMTrT1WR0hOcOa6nGkpyidFGyL1wZBF4AgbCsh7jQ/tiYm1oQh8lRFBvm4R9S
 dJpB7rX92QCEvcfROlXBUCGmbYxl4L8SFdqdlS7HjGLi5YYI9JeT4FylAd2r0Ehm
 HSuuG0Xlvedn34WDTrYOMoLl28Lha4tHl8WKaY4XqnNJ9JNqEvqRdVs/f5LcY6ja
 PTo4M8x51ZqZKE8WXtJeHQF0kA8I5i65TA/N/56tnCidVciM6oornTocsxMdMsW4
 y2NvsgS+vkMh+AhkaHR+rTu+FsbRwlWdd16NtLSNS4Q6H8n6+K8nzyB7q9D0f2uO
 1EODUQrM+08LRKL1tQJQSyhMcj13X8p9BqEYmAaGU8TrNRHx1+Yg+5iyTrAOSPqQ
 7oQTCojdrkE0MEQVYTvakZJs7yFYMpuO8Nq97DJg5++ntutUU0dMt0LNqfN/3xkY
 GdUodrb/lKIJSZAquFtm0zwjKW3d7qLWA9WWblLNz5yvJHSm9LSoEAe4gYnNfv1v
 VEHwjp5u0aWWWFE8Vwh6c5SeOZNbjfR9xwxdQJ3SJS59RHuWsv/A1CNR4P77Ow86
 Z4SfMJDxh7QQJ5wQnyIJdL0ESgPqDcK0yiWPzuQTuBweadlTNpOKApr8E29lfmX4
 jkRDcEDELwQEFqb2cISzo6Y+lC6aJvRnZnbaCb6zj3IGbpqFD8PXwiwTkV9yGfSY
 WkEuimQc62l2QnArATR8WHP6XBSyamS/zEoEuPPFgG38p9YHf75CB4lgXpTsNyJv
 4NxzPP4MaXpM0ESjLZVXx2TM8LA79fF2OPLap3y2I3ayodAMMalSpWhz6srRIKZ5
 b2omKyVFKlVeYzwRB+EjioKG1JrM443efyS9GEZhC75XEKuQ9mio6A2uSOg8fe4O
 h3JPTfwcXKWMmcwg3rhptmCAQNkzrNFBa3k62/mkf3fzilJHsr3Koby3G73lsgww
 8AISMEeh8nOG4k60CoqMQisG6vOG8fUaI8migwcipbbmbIcDU4+v4A9zPVO+gBDk
 c5zF1qMrvCITsofX5LBg99e59+I23htk/rOBae+ShSyetYx7BC1jM7qLSz364lnV
 PEHEeCKOCJlLQYMdOTM3dOExcb2YI3bnG7vqKShCSwvytiidVb1O95uUCP1/rOH0
 jkW20EPsRqp3qW2RqSaDwhvMpfl/cFnR9dZmkHjM4ApGhmGEUGuI9WGHUw/S6EN9
 NTR/POdrKwRKs9rKAlccHywFOMttXWbtSTHpDPWBA38YKsOMFUc61JHSYuPd1kLV
 EOImT8cymuMhAivtImipd17CY4DND6XhEpTFtm8Opb5a9P+fK3paKlqPlT23n6BM
 83FIPPqQT4y5buD9hCy+bf3CkAcBVRAUfOyPaVWvXR+Ihcl6UhTXSOyzXI/2JNmX
 lfTOtpVgJImrr0/jn/5yrNxJNoeVsAVv4gP7YarWBALcnn3QU2FLDKCAVG5OtSCr
 XkQ6hy5Sr4CdPbHU0SDRtjfiS1vu44Gdi9Uck/wMi9D6LFIq1k0OG2ozXPSgiN/S
 cvHTlF9oD/E2DyjABM4IbVL+KutEx27/jBN9sZJlPGhVibBlQfSnTkFqk2sC80rB
 fF9i0QxiqmT+bZ4iK8AHsEcbc2bv4ia3qxUMy6wPUmQx0MFqtmf+B9MauJH88qnQ
 LjfrmTm0KStILsux3cPTCIXzQw4at9E7Dihxoti973Buh9HX2lxLeHCuiuxmUmtO
 XAwshmHDC7W/GLq5uN3k4gq9pa5N5RA+W0rM4iUiRq//GkFUwDo9xFGD+lKK4D6H
 YqZ/cQwnLNOKtwyCIyL2fBgj/7TtaIy5PhQJDvPvcpKBSUzUGzcqnCWgy9v/Ma4z
 pShhe8G7UDPTiYWI5iYM5SuUanskHnndU2hCdOoD/ZMkoyKvKbQ2/AyinXuA02v9
 jyrFRbbHFRV/Me/x+DUn7dJpXRsfC9lmtBV1Ev6K9nJGJU4tcYkAwxkNrFEW9q9f
 q8n5/jkDUHvQGgEXvKBJCRPJ+HC+kninVv4+eSeWAWGqRFQaedCqPVdxXprqfSvI
 LbxhAMW6XspZPREqw8HLxgZD9siKXEYf26X0FLjX3INoolHedQvWD1fn2DgcZAsJ
 R2714XbSnjErB4/wkw/4bQOzvu2fSBEI7Cnf7WbZzv8gft9r8sNJ7xc1F7WZtbkS
 CrMmTojBA7av5aS8csHiqpxpEaCgFARtiapobhnqElEMRAfwXej9fXVHmlDgeamB
 Zt8pIOYrjNjj3eDjcA4hI5iuRh+/UG3djIG7sCx2COFvbRiAMfXw+WJaMyhMWgzi
 b8R4LF+a2mc6TOCZFaP2ByfOw5T/uz3pL0pw5+fn44+W5fND2UQ1fOS7KDcafQMd
 8ca+pzpm1EBLglbKVdDCu3Zj3JbwdibJz3447v6G5dPPY4JZIoH8FLLXbjQzSWil
 qYCPcjo26QgmuxyJdCHVpWSMUFzUEXgXtCmaFDjvp5uGReL09WlBSGa+GSe0plDW
 dCtTs2VCGqLA45yorjZXKDPSFjS3iQmpFQDkldlAoo4w/d+PKIyy52UiBawXuX96
 sBrxes+jTT8ho2azOAy0BfY+TJwg94YUhrBwksmzUHUaQA7gUcROIwpvXDQqLLLi
 YPTHaqhplKPRbRHGYOy4PHougSEf2oUsieDxG6XfVTjh3zXCgBST3/zlnFSLXxFD
 l8TPdu0UNm1+n9amgfS7c1a0xu/P1WO8cjZOBSZqeN2ehkn6mLSR6wqj+eVZxe4G
 x0cDXTC6+13kBJG81RVqfxUWBOOwAg56rFKLhkMdbhhrPWtcwYU+dEagkz9oG/YA
 gQ0AAz8csvTSS3LR723k+PePv+7EKhnGKgNDKJUVSfaz9bf9jXXqF6w0ytimvdbU
 lStm/jbgkRKhrqQW6DfLPd9PviqwsaiqMhmf/HHhCDv1NAshVuV8G23FUYCoYJz4
 KlDyXn02OqSsqx4JhRBRQyo2h1+CVSaEuaK4QyAQfWKhwYxeqrkI9oy5pq3uKJGJ
 kDdDbJfFRDk08FQ5NuZ70zCQGiYM86YYRSLz+5dXYUzwZ/vsRYrDx1XCNqMnz3Y5
 vUyYw7EtKP4XTlWK6ZfIJ+NhO4wifHVxQGmXbvjVBD2VEb1nJSZHg9ojzLqrdcNx
 ckTsfV/YzIkUO0ENmRda2neEQFTGEV6NtSGIC77tHmygkLJ5IqKBQOvjZ+mhgehp
 jwQT+MmRc9iItYjUnT/pZgz85WNOS526KP/VKIF5abReAg6+xJ3GVYxpjO6qMjWi
 SXWoB3PAIyhCxrgVc2tHKcGUT9QmnUCahyG//QqzPmROxauv0V8R8DdLhaB+UlIZ
 Dxv4HQTZwdcqWntqILJHpw8YH7NLPT2VaqAUtsCpzkMLQS4ZQH+kQv6LlcmgSWOs
 1SDgII8h2oNrGG4/pjMv4N1s43a4E5AmFMFDM4Cv+P8I0MzwhagOFS9d+s3144XF
 1jdshZ9X/bemjnv2nK+84bued6t28+te8BQdGIKYvajjlpmcB/xV6hrbkIrStxty
 tuThisoMY98yv3RUBU63J4hUOINNzllHWDubRvzBD3eZeEeaW5tEeGL9/aUrNmt3
 3TgYlmIvm1wBM6kzZ8JH+RS/yf3TXGimNLwTtiBEVHnbWIbqLhvIsCpKB37n9UNo
 c8dLxNRPWt9G+XDRvi43VB4d54K7BETfs2fDz12pI/WlDaqxBZyHlnjpo3R1410T
 NmzkjBrBENr9RRzuSNjkLvDNXxr969VTgVvILkp8cH5IewaMiwdTqWApzSkLQ5Gq
 LNqKOnDGIMx4v5RPQrhI6s9RqhFHthQsS8hM+VMuOSGoiC0aYK3s6TpN536n/yIP
 jjPMDhqBk/DWBO+N7kg/zeF8KvMi4dT3lJc0wMiYAwtqA8gHyZNYZhpg+eyjzswZ
 F7CDW0++NDsHjkrVoQbdyuGAbXg1tfo9PHagLJcI1O/n2lYSSXHlsxCxps6ih8WV
 Wbu+07JCGQ+AEfGrqSOHBF5mtj/iWAH3xQChg6atLw3uem39XUIkYnr+XC4bMmRD
 U8z97xeIFrmd7JhbxtKi5AoHlDrfRkSn2ZGD1o1ouGAwhRpJUpNjcMv55UzokyCe
 cEUSsg5ElfK2f6+5ZyVWHPsJ9FxfjDM/AUyQlViveJxRCSz2LpJ63hTa9LyTBhqW
 QZQ6775FHtW837lrmheRFoXp27ak8y4JnRX1VPIpqAzwxaVg/oX+jkDJkUce4jAV
 2dDKYz+VnjIv82Farx9wMaW3K5y5zH5QDr+x15oYmez8Sb2wxmGTwZfNP5eZagmd
 hP5NX5Q21qUk3XX1SDnXtIRIjgLIJrygfU6hgLdYTPX44Tnc8JtW6TQpEbuaFy5J
 TCqzVuxzoWzyEDK1DKZPINxSs2GsLIadV7Q4Wkk9beK6RwchMmNPQJBmQZ7uJhO0
 D0PvksiYV2bTLojuhW1IYUwe3xHroCLOzdPItH/Lq0eCSpLtXifUC1xtcPmZLCrB
 SwOU0/bwK82CVoKcHlhOkzmSP/KvE4CmYrYvuOPnv47gfZcN1EkmDEBYTTsGI4FS
 fjtwcGjEiAJTuCIySWJaJwpIdJ7wmjgXbi1kkHcTYqRTXLqhWiesF2omap/Nzcmx
 6fY43ANcIBMOeFGO5pcRfpLzX611icSeulNRQlFqb+OQtFeYRHc7/I6oYsguOvmZ
 j6JTQzQOND25+jcv1WsW3myBjryHgbm1UlTcxH4xwiczNXoHJ8va+g+SBjP+oux9
 B+KZTpBHG9YfkPFOHUMTd42lZtO8G9KloJchIPfZdNiheIIHQGd3cX1rzhNrxTK/
 1725LUSciSacXRp3wD03zMkGdXXfn/iv0shpayxfBpbABuM3vDjOLD9bA8D2jfTN
 MJe9pyb9ek6y8yctWncA9Ii2mYcC438yL+0LZ+SD1PVyOB2m5xQ0Gq3YXvJOvtIU
 a1wEfTEn8DddBJNk6jn1RAIwJzuC2Doy1O5Wskybwh6UXzbeV8zuP+7q/wupn1Ox
 yLTXN/pB+lAu9C6vYnW52nQrE9nZLO3Kwh8/yU8uUDYs0i4qeUwMCeQWjn5nXc05
 gNDz+Zb9bKCS60azbldfCF01MOixCWNVGeSb66Zz2asSKCWj7DHQ4LedTRZzoZeR
 =aq7v
 -----END PGP MESSAGE-----
EOF


### Create your custom build server configuration

Anybody who uses the secure build server feature of HPVS will use the same Secure Build Server image, but that image is deployed to that person's individual HPVS instance, and it is customized through a configuration file,so that your Secure Build Server can only build the application you specify, and push it to the registry that you specify, in the configuration file below.  Don't run it yet!  You have to tailor a few settings in the code cell below before you run it:

1. For the value of the "DOCKER_REPO" key, replace `<YOUR-CR-NAMESPACE>` (including the angle brackets) with the name of the IBM Container Registry namespace that you created earlier.  (Do not include the angle brackets when you substitute, they are only here to help point out which values you need to supply).

2. For the "DOCKER_PASSWORD" key, replace `<YOUR-IAM-API-KEY-VALUE>` with the API Key value you created earlier in the lab.  

3. Use the same API Key value for the value of the "DOCKER_RO_PASSWORD" key

Once you have made these three substitutions in the below code cell, run the code cell.

In [ ]:
%%bash
cd secure-build-cli
cat <<EOF > sbs-config.json
{
    "CICD_PUBLIC_IP": "",
    "CICD_PORT": "443",
    "IMAGE_TAG": "1.3.0",
    "CONTAINER_NAME": "SBContainer",
    "GITHUB_KEY_FILE": "~/.ssh/id_rsa",
    "GITHUB_URL": "git@github.com:IBM/secure-bitcoin-wallet.git",
    "GITHUB_BRANCH": "master",
    "IMAGE_TAG_PREFIX": "s390x-v1",
    "REPO_ID": "sbs",
    "DOCKER_REPO": "bjs-secureimages/secure-bitcoin-wallet15",
    "DOCKER_USER": "iamapikey",
    "DOCKER_PASSWORD": "redacted",
    "DOCKER_RO_USER": "iamapikey",
    "DOCKER_RO_PASSWORD": "redacted",
    "DOCKER_CONTENT_TRUST_BASE": "False",
    "DOCKER_CONTENT_TRUST_BASE_SERVER": "",
    "DOCKER_BASE_SERVER": "us.icr.io",
    "DOCKER_PUSH_SERVER": "us.icr.io",
    "DOCKER_CONTENT_TRUST_PUSH_SERVER": "https://us.icr.io:4443",
    "ENV_WHITELIST":  ["ZHSM", "APIKEY", "INSTANCE_ID", "IAM_ENDPOINT"],
    "ARG": {
      "NO_GRPC_BUILD": "1"
    }
  }
EOF
cat sbs-config.json


You can run the code cell below to display the contents of the file you just created to ensure it looks good.  If you made a typo, fix it and rerun the above code cell.

In [ ]:
! cd secure-build-cli && cat sbs-config.json

Secure communication between you and your secure build server is obtained by using mutual Transport Layer Security (TLS). TLS is the same security protocol that protects most Web traffic today, but most Web interactions only require the server to be authenticated.  Mutual TLS refers to the situation where not only does the client want to ensure that the server is legitimate, but the server also wants to authenticate the identity of the client.  Your Web browser is constantly receiving server certificates and verifying them while you surf the web. Client certificates are what are used when the server wants to verify the client.  

Create the client certificate and client certification authority (CA) by running the following command:

In [ ]:
! cd secure-build-cli && ./build.py create-client-cert --env sbs-config.json

The next code cell will display the client certificate and the client certification authority in base64-encoding.

*Note:* Keep the output from the below code cell.  You will be copying most of its output into your clipboard for a subsequent step.

If you already know about base64-encoding, go ahead and run the cell.  If you don't, read on.  

Base64-encoding is a means of encoding any data, including binary data, into human-readable characters.  It takes groups of 6 bits, converts these six bits into a number from 0 to 63  (2 to the 6th power is 64), and replaces these 6 bits into one of 64 human-readable characters.  What 64 human-readable characters were chosen?  lowercase alphabetic characters a-z for 26, uppercase alphabetic characters A-Z for 26 more, the digits 0-9 for 10 more, which gives us 62.  Then `+` and `/` were chosen to make 64!  It should be noted that base64-encoding should not be confused with encryption.  It is easy to convert a base64-encoded string back to its original format.


In [ ]:
! cd secure-build-cli &&  ./build.py instance-env --env sbs-config.json

Run the following to command to show that you do not currently have any HPVS instances:

In [ ]:
! ibmcloud hpvs instances

In [ ]:
! ibmcloud hpvs instance-create SBContainer lite-s dal13 --rd-path "secure_build.asc" -i 1.3.0 \

# copy and paste, in the blank line above me, the super long -e CLIENT_CRT=...  -e CLIENT=... from the output of the command \ 
# a couple cells above.  you can leave me here because I'm just a comment.

It will take a few minutes for your instance to be built. Copy the entire `ibmcloud hpvs instance crn:...` command above, which has your unique instance id (shown in the output of the above command, it starts with _crn:_) into the code cell below and run it periodically until the instance is successfully created.

In [ ]:
! 
# copy and paste above the entire 'ibmcloud hpvs instance crn:.....' commmand from the \
# output from the previous command, and run it periodically (every minute or so) until it is running.

The code cell below will show the current state of your secure build configuration file.  The key thing to note from the output is that the IP address value of the "CICD_PUBLIC_IP" key is blank.

In [ ]:
! cd secure-build-cli && cat sbs-config.json

Now that your secure build server instance is running, make a note or copy the IP address shown in the `Public IP address` field from the command output from two cells above (the `ibmcloud hpvs instance crn:...` command). The code cell below uses an awkward-looking, yet simple `sed` command to add the IP address in the proper spot.  Replace the `xx.xx.xx.xx` string below with the Public IP address of your secure buil container, then run the code cell:

In [ ]:
! cd secure-build-cli && sed -i -e 's/\"CICD_PUBLIC_IP\"\: \"\"/\"CICD_PUBLIC_IP\"\: \"xx.xx.xx.xx\"/' sbs-config.json

Run the code cell below to repeat the command that displays your configuration file, to ensure that your public IP address was added properly:

In [ ]:
! cd secure-build-cli && cat sbs-config.json

Run the following command to check the status of your Secure Build Server instance. The status field in the output should be empty:

In [ ]:
! cd secure-build-cli && ./build.py status --env sbs-config.json --noverify

Run the following code cell to get the server certificate signing request (CSR):

In [ ]:
! cd secure-build-cli && ./build.py get-server-csr --env sbs-config.json --noverify

Use the following command to sign the server CSR:

In [ ]:
! cd secure-build-cli && ./build.py sign-csr --env sbs-config.json

To post the signed server certificate to your Secure Build Server instance, run the following code cell:

In [ ]:
! cd secure-build-cli && ./build.py post-server-cert --env sbs-config.json --noverify

To check and verify the status for your Secure Build Server instance, run the following code cell.  Note that the status says _restarted nginx_. This is because the nginx server running inside the Secure Build Server was restarted in order to use the server certificate you just posted.

In [ ]:
! cd secure-build-cli && ./build.py status --env sbs-config.json

Run the following code cell to initialize the configuration for your Secure Build Server instance. The status of the output should be `OK`:

In [ ]:
! cd secure-build-cli && ./build.py init --env sbs-config.json

The code cell below will start the build of your application image. The command returns quickly, as soon as it submits the request to start the build.  You status message from the command should say `OK: async build started`, which indicates that the build of your image is starting up on your Secure Build Server container

Run the below code cell to start the your build:

In [ ]:
! cd secure-build-cli && ./build.py build --env sbs-config.json

Check the status of your build by running the code cell below.  You can check it periodically, your build will take about ten minutes.  You can run this code cell periodically, as well as the command in the subsequent code cell which displays the logs from the build.  For now, check the status by running the code cell below.  Your build has completed successfully once the output of the code cell has the value of *success* in the *status* field:

In [ ]:
! cd secure-build-cli && ./build.py status --env sbs-config.json

Your secure image build will produce a lot of output in its log. You can check the log of your build by running the below code cell.  You can clear the output of this command from your Jupyter notebook by selecting *Cell-> Current Outputs-> Clear* from the menu while the code cell is selected.  (The build logs are retained on the Secure Build Server and can be retrieved at any time).

In [ ]:
! cd secure-build-cli && ./build.py log --log build --env sbs-config.json

Run the code cell below to download the state image (the backup of your state) of our Secure Build Server. This can be used to recreate your Secure Build Server if necessary. This backup also contains the logs of your Secure Build Server.

In [ ]:
! cd secure-build-cli && ./build.py get-state-image --env sbs-config.json

The Secure Build Server creates a signed manifest file for each successful build. The manifest file is useful for audit purposes. You can verify the source and integrity of the build and the built image or you can pass the manifest file to an auditor to do so. Run the below code cell to download and verify the manifest file:

In [ ]:
! cd secure-build-cli && ./build.py get-manifest --env sbs-config.json  --verify-manifest

The above command downloads the manifest file and verifies the digital signature on it.  In the last line of output from the above code cell you should see *verify=OK* which indicates that the digital signature is valid and is proof that the manifest file has not been tampered with.

In order to create an HPVS instance using the application image you just built, you must download an encrypted registration file for your application container image that was created by the build. This registration file can only be decrypted, and thus used, by your application container image.

Note the value chosen for the *--key-id* argument:  *secure-build-369f09c-1*.  This value _369f09c_ was chosen as this is a hash of the image we built and is appended to the end of the tag name. 

If you look at your command output a few code cells above where you checked on your build status, you can see this hash at the end of the value of the *image_tag* key, e.g.,  `"image_tag" : "s390x-v1-369f09c"`. 

The *--key-id* value can be anything, but putting this hash value as part of the name helps to keep from confusing different keys if you have multiple keys.

Copy the command in the code below (everything but the leading exclamation point) and run it in your terminal tab. As part of this command, you will be prompted to enter a passphrase three times- the first two times to set the passphrase, and then the key is used to sign the registration file, so you are asked to enter the passphrase you just set.


In [ ]:
! cd /home/jovyan/work/ibm-cloud-hpvs-sbs-byoi-lab/secure-build-cli && ./build.py get-config-json --env sbs-config.json --key-id secure-build-369f09c-1

Run the code cell below to create your HPVS instance using your BYOI image that you just built.  The value for the `-i` argument must be the value of the *image_tag* key in the output of the command that checks build status.

In [ ]:
! cd secure-build-cli && ibmcloud hpvs instance-create securewallet lite-s dal13 --rd-path sbs.enc -i s390x-v1-369f09c

It will take a few minutes for your instance to be built. Copy the entire `ibmcloud hpvs instance crn:...` command above, which has your unique instance id (shown in the output of the above command, it starts with crn:) into the code cell below and run it periodically until the instance is successfully created.  The instance is successfully created when the value of the _Cloud state_ field of the command output is _active_.

In [ ]:
! 
# copy and paste above the entire 'ibmcloud hpvs instance crn:.....' commmand from the \
# output from the previous command, and run it periodically (every minute or so) until it is running.

Once your instance is active, note the value of the _Public IP address_ field in the output.  

Double-click here to go into _edit_ mode of this markdown cell.  In the line below, change `x.x.x.x` to your public IP address.  The actual URL is on the right side of the line (between parentheses). The text of the link is on the left side of the line in between square brackets.  You may as well change both occurrences of `x.x.x.x` for clarity, but the one that matters is the link on the right side.   (If you don't see both sides you haven't succesfully got into edit mode-  The cell has a green vertical line at the left when you are in edit mode, a blue vertical line when you are not in edit mode.  After you have made the change, run this markdown cell and it will present you with the properly formatted link. Click the link and you will bring up your sample secure bitcoin wallet application.

[Change the 52.116.0.227 to your public IP and then click here](https://52.116.0.227/electrum)

## Running the sample application

In order to test your securely built sample _Secure Bitcoin Wallet Application_, follow the instructions at [this link](https://ibm-wsc.github.io/hyper-protect-virtual-servers-workshop/securebuild-lab/bitcoin_wallet/), and follow the instructions on that page.  When you are finished with that page, return here so that you can clean up from your lab activities. 

## Clean up when you are done with the lab

You will want to clean up (i.e., delete) the following artifacts that you created for this lab:

1. The IBM Cloud IAM Key you created for this lab
2. The ssh key you created for this lab
3. The Secure Build Server HPVS instance you created
4. The sample Secure Bitcoin Wallet Application HPVS instance you created
5. The Secure Bitcoin Wallet Application container image that you created for this lab
6. The IBM Container Registry namespace that you created for this lab


### Delete your Secure Build Server HPVS instance

Run the code cell below to list your HPVS instances:

In [ ]:
! ibmcloud hpvs instances

Find your instance named *SBContainer*, and copy its CRN (_Cloud Resource Name_) and paste it at the end of the `ibmcloud hpvs instance-delete ` command in the below code cell and then run it.

In [ ]:
! ibmcloud hpvs instance-delete --force     # <- paste your full CRN name to the left of the pound sign

### Delete your sample Secure Bitcoin Wallet Application HPVS instance

Run the code cell below to list your HPVS instances:

In [ ]:
! ibmcloud hpvs instances

Find your instance named _securewallet_ and copy its CRN value and paste it at the end of the ibmcloud hpvs instance-delete command in the below code cell and then run it.

In [ ]:
! ibmcloud hpvs instance-delete --force crn:v1:bluemix:public:hpvs:dal13:a/1e963a246cc69a44df65e277e14239d5:a3e747b1-d11c-4e0a-9d9c-6311aa59345f::   # <- paste your full CRN name to the left of the pound sign

### Delete your container image

Run the following code cell to list your IBM Container Registry images:

In [ ]:
! ibmcloud cr images

Copy and paste the Repository name (first column) of the image you built for this lab, and paste it into the code cell below after `ibmcloud cr image-rm ` and then run the code cell.  This will remove your image.

In [ ]:
! ibmcloud cr image-rm    # <- paste your repository name after 'image -rm '

You can rerun the `ibmcloud cr images` command to verify that you deleted the Secure Bitcoin Wallet Application image:

In [ ]:
! ibmcloud cr images

### Delete the IBM Container Registry namespace that you created for this lab

Run the following code cell to list your IBM Container Registry namespaces:

In [ ]:
! ibmcloud cr namespaces

Copy and paste the namespace you created for this lab to the end of the code below and then run it.

*Note:* If you used an existing namespace and wish to keep it, skip this step.

In [ ]:
! ibmcloud cr namespace-rm --force         # <- paste the name of the namespace you want to delete